In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
tf.keras.backend.set_floatx('float16')

In [ ]:
VOC_data_path = "../data/VOCdevkit/VOC2012/"
preprocessed_data_path = "../data/preprocessed_data/"
saved_weights_path = "../data/saved_weights/"

In [2]:
from data_management.VOC2012ManagerObjDetection import VOC2012ManagerObjDetection
db_manager = VOC2012ManagerObjDetection(VOC_data_path,
                                        batch_size=32, floatType=16)

In [3]:
from models.SSD300 import SSD300
SSD300_model = SSD300(21, floatType=16)
SSD300_model.build(input_shape=(32, 300, 300, 3))
SSD300_model.load_vgg16_imagenet_weights()

In [4]:
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay

init_lr = 0.001
lr_decay = PiecewiseConstantDecay(
    boundaries=[80000, 10000, 120000],
    values=[init_lr, 0.0005, 0.0001, 0.00005])

optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_decay,
    momentum=0.9)

In [5]:
from data_management.preprocessing import *
# Preprocess data
# to be done only once,
# if the path exists: assuming that the preprocessing has already been done completely
if not os.path.exists(preprocessed_data_path):
    saveGTdata(VOC_data_path, preprocessed_data_path)

In [6]:
from data_management.preprocessing import *
imgs, confs, locs = loadGTdata(preprocessed_data_path, nb_data_to_load=100)

100%|████████████████████████████████████████████████████████████████████████████████| 374/374 [00:04<00:00, 93.44it/s]


In [7]:
# if a model was already trained
#SSD300_model.load_weights(path)

In [ ]:
from utils.train import *
train(SSD300_model, optimizer,
      imgs, confs, locs, weights_path=saved_weights_path,
      num_epoch=2, inter_save=1)